Import Necessary libraries

Team Title : PNT2022TMID29441

Project Title : A novel method for handwritten digit recognition system

Model Training

Importing Packages

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import keras
from keras.models import Sequential
from keras.layers import Conv2D, Lambda, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import BatchNormalization

from keras.preprocessing.image import ImageDataGenerator

from keras.utils.np_utils import to_categorical

from keras.datasets import mnist

# Load the data

In [ ]:
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
print(X_train.shape)
print(X_test.shape)

11501568/11490434 [==============================] - 0s 0us/step
(60000, 28, 28)
(10000, 28, 28)


# Data Pre-Processing

In [ ]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [ ]:
X_train = X_train.reshape(-1,28,28,1)
X_test = X_test.reshape(-1,28,28,1)

In [ ]:
Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)

In [ ]:
mean = np.mean(X_train)
std = np.std(X_train)

def standardize(x):
    return (x-mean)/std

# Create model

In [ ]:
model=Sequential()

model.add(Conv2D(filters=64, kernel_size = (3,3), activation="relu", input_shape=(28,28,1)))
model.add(Conv2D(filters=64, kernel_size = (3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())

model.add(Conv2D(filters=128, kernel_size = (3,3), activation="relu"))
model.add(Conv2D(filters=128, kernel_size = (3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())    

model.add(Conv2D(filters=256, kernel_size = (3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
    
model.add(Flatten())
model.add(Dense(512,activation="relu"))
    
model.add(Dense(10,activation="softmax"))
    
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 64)        640       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 64)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 12, 12, 64)       256       
 ormalization)                                                   
                                                                 
 conv2d_2 (Conv2D)           (None, 10, 10, 128)       73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 8, 8, 128)         1

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        rotation_range=15,
        zoom_range = 0.01,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=False,
        vertical_flip=False)


train_gen = datagen.flow(X_train, Y_train, batch_size=128)
test_gen = datagen.flow(X_test, Y_test, batch_size=128)

# Train the model

In [ ]:
epochs = 10
batch_size = 128
train_steps = X_train.shape[0] // batch_size
valid_steps = X_test.shape[0] // batch_size

es = keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=10,
        verbose=1,
        mode="max",
        restore_best_weights=True,
     )

rp = keras.callbacks.ReduceLROnPlateau(
        monitor="val_accuracy",
        factor=0.2,
        patience=3,
        verbose=1,
        mode="max",
        min_lr=0.00001,
     )

In [ ]:
history = model.fit(train_gen, 
                    epochs = epochs, 
                    steps_per_epoch = train_steps,
                    validation_data = test_gen,
                    validation_steps = valid_steps, 
                    callbacks=[es, rp])

Epoch 1/10
468/468 [==============================] - 351s 748ms/step - loss: 0.1261 - accuracy: 0.9603 - val_loss: 0.2915 - val_accuracy: 0.9031 - lr: 0.0010
Epoch 2/10
468/468 [==============================] - 353s 755ms/step - loss: 0.0555 - accuracy: 0.9830 - val_loss: 0.0458 - val_accuracy: 0.9857 - lr: 0.0010
Epoch 3/10
468/468 [==============================] - 354s 757ms/step - loss: 0.0433 - accuracy: 0.9872 - val_loss: 0.0378 - val_accuracy: 0.9890 - lr: 0.0010
Epoch 4/10
468/468 [==============================] - 351s 750ms/step - loss: 0.0373 - accuracy: 0.9884 - val_loss: 0.0353 - val_accuracy: 0.9894 - lr: 0.0010
Epoch 5/10
468/468 [==============================] - 355s 759ms/step - loss: 0.0346 - accuracy: 0.9894 - val_loss: 0.0454 - val_accuracy: 0.9873 - lr: 0.0010
Epoch 6/10
468/468 [==============================] - 353s 754ms/step - loss: 0.0300 - accuracy: 0.9907 - val_loss: 0.0311 - val_accuracy: 0.9903 - lr: 0.0010
Epoch 7/10
468/468 [==========================

# Save the Model

In [ ]:
model.save("mnistcnn.h5")

In [ ]:
!tar -zcvf model.tgz mnistcnn.h5

mnistcnn.h5


# Install necessary packages(IBM WATSON)

In [ ]:
!pip install watson-machine-learning-client

     |████████████████████████████████| 538 kB 22.8 MB/s eta 0:00:01


In [ ]:
!pip install ibm_watson_machine_learning

In [ ]:
from ibm_watson_machine_learning import APIClient



credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": "8ytCzKXJ97bxcr94wFxe3-6OvAzikCyCEYAupWLbFiwb"
}

client = APIClient(credentials)
client

In [ ]:
client.spaces.get_details()

{'resources': [{'entity': {'compute': [{'crn': 'crn:v1:bluemix:public:pm-20:us-south:a/9c85d3e704a74497ae0f6d94b3ca1507:3233cca6-6557-429d-8a8f-ddd4e72a176a::',
      'guid': '3233cca6-6557-429d-8a8f-ddd4e72a176a',
      'name': 'Watson Machine Learning-3w',
      'type': 'machine_learning'}],
    'description': '',
    'name': 'Handwritten-Digit-Recognition',
    'scope': {'bss_account_id': '9c85d3e704a74497ae0f6d94b3ca1507'},
    'stage': {'production': False},
    'status': {'state': 'active'},
    'storage': {'properties': {'bucket_name': '74b406c9-cc60-4761-9971-7ae2690d0f6f',
      'bucket_region': 'us-south',
      'credentials': {'admin': {'access_key_id': '9c3fda754b45470ca7a06af71b62a8c5',
        'api_key': 'lConSvSsUps-elQlHk3hJZ7jbGOBAjZaMkF9PIsJssMM',
        'secret_access_key': '5fcab02cf396d04f50520fefc0a3195c653824e0c72b6896',
        'service_id': 'ServiceId-66763527-1a55-4ee9-a9bf-7d7686697cc6'},
       'editor': {'access_key_id': '69e2fe5e7c674d83bdc04c78f1c381d7',

In [ ]:
def guid_from_space_name(client, ibm_deploy):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']['name'] == ibm_deploy)['metadata']['id'])

In [ ]:
space_uid = guid_from_space_name(client, 'Handwritten-Digit-Recognition')
space_uid

'69eb14d9-dd12-410e-a848-fc25becb11e5'

In [ ]:
client.set.default_space(space_uid)

'SUCCESS'

# Define model specifications for deployment 

In [ ]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [ ]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_spec_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [ ]:
model_details = client.repository.store_model(model="model.tgz", meta_props={
    client.repository.ModelMetaNames.NAME: "CNN",
    client.repository.ModelMetaNames.TYPE: "tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid
})

model_id = client.repository.get_model_id(model_details)

In [ ]:
model_details

{'entity': {'hybrid_pipeline_software_specs': [],
  'software_spec': {'id': 'acd9c798-6974-5d2f-a657-ce06e986df4d',
   'name': 'tensorflow_rt22.1-py3.9'},
  'type': 'tensorflow_2.7'},
 'metadata': {'created_at': '2022-11-17T06:27:12.604Z',
  'id': '7f19ab54-adf1-49ff-89bf-bff32fc47632',
  'modified_at': '2022-11-17T06:27:16.460Z',
  'name': 'CNN',
  'owner': 'IBMid-668000ETK4',
  'resource_key': 'bf16aa81-1bdc-404f-ad45-66cd6b539292',
  'space_id': '69eb14d9-dd12-410e-a848-fc25becb11e5'},
 'system': {'warnings': []}}

In [ ]:
model_id

'7f19ab54-adf1-49ff-89bf-bff32fc47632'

# Download the deployed model

In [ ]:
client.repository.download(model_id, "model.tar.gz")

Successfully saved model content to file: 'model.tar.gz'


'/home/wsuser/work/model.tar.gz'

In [ ]:
ls

mnistcnn.h5  model.tar.gz  model.tgz
